In [1]:
import scipy.sparse as sp
import networkx as nx
import pandas as pd
import numpy as np

In [3]:
from dataclasses import dataclass
import networkx as nx
from typing import Sequence
import itertools

@dataclass
class Triangle:
    indices:Sequence[int] # 3 indices in ascending order
    pos_weights: Sequence[bool] # whether three edges (0-1, 1-2, 2-0) are positive
    
    def __hash__(self):
        return hash(tuple(self.indices))
    def __eq__(self, other):
        if isinstance(other, Triangle):
            return self.indices == other.indices
        return False
    def __init__(self, idx_0:int, idx_1:int, idx_2:int, G:nx.Graph):
        self.indices = sorted([idx_0, idx_1, idx_2])
        
        edge_with_data = G.edges
        self.pos_weights = list()
        self.pos_weights.append(edge_with_data[(self.indices[0], self.indices[1])]['weight'])
        self.pos_weights.append(edge_with_data[(self.indices[1], self.indices[2])]['weight'])
        self.pos_weights.append(edge_with_data[(self.indices[2], self.indices[0])]['weight'])
        
def is_unbalanced_triangle(t:Triangle):
    # 1 or 3 edges are negative
    return t.pos_weights[0] * t.pos_weights[1] * t.pos_weights[2] < 0
    
def explore_triangles(G:nx.Graph):
    triangle_set = set()
    
    def find_triangle_from_vertex(G:nx.Graph, start_idx:int):
        adj_nodes = [n for n in G.neighbors(start_idx)]
        
        # get all pairs
        for a, b in itertools.combinations(adj_nodes, 2):
            # if a, b are connected
            a_neighbors = G.neighbors(a)
            if b in a_neighbors:
                # add to set
                triangle_set.add(Triangle(start_idx, a, b, G))

    for start_idx in G.nodes:
        find_triangle_from_vertex(G, start_idx)
    
    unbalaneced_num = 0
    for t in triangle_set:
        if is_unbalanced_triangle(t):
            unbalaneced_num += 1
    
    # print statistics
    print(f'All triangle num: {len(triangle_set)}')
    print(f'Unbalanced triangle num: {unbalaneced_num}')
    
    return unbalaneced_num, len(triangle_set)

# for general graph
def explore_triangles_general(G:nx.Graph, b_complete_graph:bool):
    triangle_set = set()
    
    
    # count negative edges
    our_num_triangle = 0
    num_unbalanced = 0
    for u,v,d in G.edge(data=True):
#         if b_complete_graph and d['weight'] > 0:
#             continue
        u_neighb = [n for n in G.neighbors(u)]
        v_neighb = [n for n in G.neighbors(v)]
        curr_weight = d['weight']
        
        intersected = set(u_neighb) & set(v_neighb)
        our_num_triangle += len(intersected)
        
        for i in intersected:
            if G.edge[(u,i)]['weight'] * G.edge[(v,i)]['weight'] * curr_weight < 0:
                num_unbalanced += 1


In [4]:
import numpy as np
import scipy.sparse as sp

def find_num_unbalanced(Ap:sp.csc.csc_matrix, An:sp.csc.csc_matrix):
    # convert to undirected graph
    Ap = Ap + Ap.transpose()
    An = An + An.transpose()
    # binarize
    Ap = (Ap>0).astype(int)
    An = (An>0).astype(int)
    A = Ap - An
    Abar = ((Ap + An) > 0).astype(int)
    
    print(f'Num node: {Ap.shape[0]}')
    
    A_cube = A.dot(A.dot(A))
    balminusunbal = A_cube.diagonal().sum()
    print(f'acube trace: {balminusunbal}')
    
    Abar_cube = Abar.dot(Abar.dot(Abar))
    baladdunbal = Abar_cube.diagonal().sum()
    print(f'abar cube trace: {baladdunbal}')
    
    unbalance_num = (baladdunbal - balminusunbal) / 2
    total_num_triangle = baladdunbal / 6
    print(f'Total triangle num: {total_num_triangle}')
    
    unbalance_num /= 6
    print(f'unbalance num: {unbalance_num}')
    print(f'unbalanced ratio: {unbalance_num / total_num_triangle}')
    
    
    return unbalance_num

In [20]:
def dataset_info(dataset, Ap:sp.csc.csc_matrix, An:sp.csc.csc_matrix):
    # convert to undirected graph
    Ap = Ap + Ap.transpose()
    An = An + An.transpose()
    # binarize
    Ap = (Ap>0).astype(int)
    An = (An>0).astype(int)
    A = Ap - An
    Abar = ((Ap + An) > 0).astype(int)
    
    num_node = Ap.shape[0]
    
    print(f'Num node: {num_node}')
    
    num_pos_edges = Ap.sum()
    num_neg_edges = An.sum()
    
    A_cube = A.dot(A.dot(A))
    balminusunbal = A_cube.diagonal().sum()
    print(f'acube trace: {balminusunbal}')
    
    Abar_cube = Abar.dot(Abar.dot(Abar))
    baladdunbal = Abar_cube.diagonal().sum()
    print(f'abar cube trace: {baladdunbal}')
    
    unbalance_num = (baladdunbal - balminusunbal) / 2
    total_num_triangle = baladdunbal / 6
    print(f'Total triangle num: {total_num_triangle}')
    
    unbalance_num /= 6
    unbalance_ratio = unbalance_num / total_num_triangle
    print(f'unbalance num: {unbalance_num}')
    print(f'unbalanced ratio: {unbalance_ratio}')
    
    print(dataset+'&'+str(num_node)+'&'+str(num_pos_edges)+'&'+\
          str(num_neg_edges)+'&'+str(unbalance_num)+'&'+str(unbalance_ratio))
    return num_node, num_pos_edges, num_neg_edges, unbalance_num, unbalance_ratio

# PPI unbalanced

In [6]:
A_p = sp.load_npz('../data/PPI/adjacency_plus.npz')
A_n = sp.load_npz('../data/PPI/adjacency_minus.npz')
dataset_info(A_p, A_n)

Num node: 3058
acube trace: 21894
abar cube trace: 23022
Total triangle num: 3837.0
unbalance num: 94.0
unbalanced ratio: 0.024498305968204327
PPI unbalanced 94.0


In [7]:
A_p=(A_p>0).astype(int)
A_p

<3058x3058 sparse matrix of type '<class 'numpy.int64'>'
	with 7996 stored elements in Compressed Sparse Column format>

In [8]:
A_p = sp.load_npz('../data/PPI/adjacency_plus.npz')
A_n = sp.load_npz('../data/PPI/adjacency_minus.npz')
dataset_info(A_p, A_n)

All triangle num: 3837
Unbalanced triangle num: 94
Two triangle numbers: 3837, 3837
PPI&11860&7996&3864&94&0.0245
---new result---
All triangle num: 3837
Unbalanced triangle num: 94
Two triangle numbers: 3837, 3837
PPI&11860&7996&3864&94&0.0245


# Rainfall unbalanced

In [9]:
A_p = sp.load_npz('../data/rainfall/plus_cc.npz')
A_n = sp.load_npz('../data/rainfall/minus_cc.npz')
dataset_info(A_p, A_n)

Num node: 306
acube trace: 12443544
abar cube trace: 28652616
Total triangle num: 4775436.0
unbalance num: 1350756.0
unbalanced ratio: 0.28285501051631723
Rainfall unbalanced 1350756.0


# wikirfa

In [12]:
A_p = sp.load_npz('../data/wikirfa/pruned_A_p.npz')
A_n = sp.load_npz('../data/wikirfa/pruned_A_n.npz')
dataset_info(A_p, A_n)

Num node: 7634
acube trace: 3558809
abar cube trace: 7465835
Total triangle num: 1244305.8333333333
unbalance num: 325585.5
unbalanced ratio: 0.26166035011489003


325585.5

# SP1500

In [ ]:
A_p = sp.load_npz('../data/SP1500/adjacency_plus_cc.npz')
A_n = sp.load_npz('../data/SP1500/adjacency_minus_cc.npz')
dataset_info(A_p, A_n)

# SSBM

In [ ]:
import pickle as pk
data = pk.load(open('../data/SSBM/10_500_500000_10_80_10_150_0_1000.pk','rb'))
A_p, A_n = data.A_p, data.A_n

In [ ]:
dataset_info(A_p, A_n)

## Finance

In [24]:
res_array = np.zeros((21,5))
for year in range(2000, 2021):
    A = sp.load_npz('../data/corr_networks/adj_MR_yearly_'+str(year)+'.npz')
    A_p = (abs(A) + A)/2
    A_n = (abs(A) - A)/2
    res_array[year-2000] = data_info(str(year), A_p, A_n)

Num node: 451
acube trace: 78434749
abar cube trace: 91520953
Total triangle num: 15253492.166666666
unbalance num: 1090517.0
unbalanced ratio: 0.07149293998282558
2000&451&187715&15528&1090517.0&0.07149293998282558
Num node: 451
acube trace: 72884479
abar cube trace: 91437715
Total triangle num: 15239619.166666666
unbalance num: 1546103.0
unbalanced ratio: 0.10145286329606991
2001&451&180943&22238&1546103.0&0.10145286329606991
Num node: 451
acube trace: 69176395
abar cube trace: 91424023
Total triangle num: 15237337.166666666
unbalance num: 1853969.0
unbalanced ratio: 0.1216727686551269
2002&451&178691&24480&1853969.0&0.1216727686551269
Num node: 451
acube trace: 58258561
abar cube trace: 91206397
Total triangle num: 15201066.166666666
unbalance num: 2745653.0
unbalanced ratio: 0.1806223964751069
2003&451&177977&25032&2745653.0&0.1806223964751069
Num node: 451
acube trace: 40188115
abar cube trace: 90857119
Total triangle num: 15142853.166666666
unbalance num: 4222417.0
unbalanced rat

In [25]:
res_array.mean(0)

array([4.51000000e+02, 1.48527762e+05, 5.43127619e+04, 4.08594262e+06,
       2.69708753e-01])